In [1]:
!unzip data.zip

Archive:  data.zip
   creating: data/
   creating: data/test/
  inflating: data/test/Groupe1_Image1.jpg  
  inflating: data/test/Groupe1_Image4.jpg  
  inflating: data/test/Groupe1_Image5.jpg  
  inflating: data/test/Groupe1_Image9.jpg  
  inflating: data/test/Groupe2_Image1.jpeg  
  inflating: data/test/Groupe2_Image11.jpg  
  inflating: data/test/Groupe2_Image9.jpg  
  inflating: data/test/Groupe5_image04.jpg  
  inflating: data/test/Groupe5_image08.jpg  
  inflating: data/test/Groupe5_image11.jpeg  
  inflating: data/test/Groupe5_image13.jpeg  
  inflating: data/test/Groupe5_image17.jpeg  
  inflating: data/test/Groupe6_image6.jpg  
  inflating: data/test/img4.jpg      
  inflating: data/test/img6.jpg      
  inflating: data/test/t3i13.jpg     
  inflating: data/test/t3i14.jpeg    
  inflating: data/test/t3i20.png     
  inflating: data/test/t3i21.jpeg    
  inflating: data/test/t3i28.jpg     
   creating: data/train/
  inflating: data/train/Groupe1_Image10.jpg  
  inflating: data/t

In [2]:
import os
import json
import cv2
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import LeaveOneOut
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error

# --- CHARGER LES DONNÉES JSON ---
json_path = "gt.json"
with open(json_path, "r") as f:
    data = json.load(f)

# Créer un dictionnaire {image_name: actual_count}
ground_truth = {item["image"]: item["actual_count"] for item in data["data"]}

# --- EXTRACTION DES CARACTÉRISTIQUES ---
def extract_features(image_path):
    """Charge l'image et extrait les caractéristiques de contours/lignes."""
    img = cv2.imread(image_path)
    if img is None:
        return None  # Évite les erreurs si une image ne charge pas
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Détection des contours
    edges = cv2.Canny(gray, 50, 150, apertureSize=3)

    # Détection des lignes horizontales
    lines = cv2.HoughLinesP(edges, 1, np.pi / 180, 100, minLineLength=100, maxLineGap=10)

    if lines is None:
        return [0, 0, 0, 0]  # Aucune ligne détectée

    horizontal_lines = [line for line in lines if abs(np.arctan2(line[0][3] - line[0][1], line[0][2] - line[0][0]) * 180 / np.pi) < 10]

    num_lines = len(horizontal_lines)
    y_positions = [line[0][1] for line in horizontal_lines]
    avg_y = np.mean(y_positions) if y_positions else 0
    std_y = np.std(y_positions) if y_positions else 0

    return [num_lines, avg_y, std_y, len(lines)]

# --- EXTRAIRE LES DONNÉES POUR LE TRAINING ---
train_folder = "data/train"

X, y = [], []
for image_name in os.listdir(train_folder):
    if image_name in ground_truth:  # Vérifier si l'image a une annotation
        image_path = os.path.join(train_folder, image_name)
        features = extract_features(image_path)
        if features is not None:
            X.append(features)
            y.append(ground_truth[image_name])

X = np.array(X)
y = np.array(y)

# --- NORMALISATION ---
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# --- ENTRAÎNEMENT AVEC LEAVE-ONE-OUT ---
loo = LeaveOneOut()
errors = []

rf = RandomForestRegressor(n_estimators=1000, random_state=42, warm_start=True)

for train_idx, test_idx in loo.split(X_scaled):
    X_train, X_test = X_scaled[train_idx], X_scaled[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]

    if len(X_train) > 1:
        rf.fit(X_train, y_train)  # Entraîner le modèle

    y_pred = rf.predict(X_test)
    errors.append(abs(y_test[0] - y_pred[0]))  # Calculer l'erreur absolue

# --- CALCUL DE LA MAE ---
mean_error_rf = np.mean(errors)
print(f"Erreur Absolue Moyenne (MAE) : {mean_error_rf:.2f}")


/usr/local/lib/python3.11/dist-packages/sklearn/ensemble/_forest.py:466: UserWarning: Warm-start fitting without increasing n_estimators does not fit new trees.
  warn(
/usr/local/lib/python3.11/dist-packages/sklearn/ensemble/_forest.py:466: UserWarning: Warm-start fitting without increasing n_estimators does not fit new trees.
  warn(
/usr/local/lib/python3.11/dist-packages/sklearn/ensemble/_forest.py:466: UserWarning: Warm-start fitting without increasing n_estimators does not fit new trees.
  warn(
/usr/local/lib/python3.11/dist-packages/sklearn/ensemble/_forest.py:466: UserWarning: Warm-start fitting without increasing n_estimators does not fit new trees.
  warn(
/usr/local/lib/python3.11/dist-packages/sklearn/ensemble/_forest.py:466: UserWarning: Warm-start fitting without increasing n_estimators does not fit new trees.
  warn(
/usr/local/lib/python3.11/dist-packages/sklearn/ensemble/_forest.py:466: UserWarning: Warm-start fitting without increasing n_estimators does not fit new t

Erreur Absolue Moyenne (MAE) : 1.74


In [5]:
# --- TEST DU MODÈLE SUR L'ENSEMBLE TEST ---
test_folder = "data/test"

X_test, y_test, image_names = [], [], []
for image_name in os.listdir(test_folder):
    if image_name in ground_truth:
        image_path = os.path.join(test_folder, image_name)
        features = extract_features(image_path)
        if features is not None:
            X_test.append(features)
            y_test.append(ground_truth[image_name])
            image_names.append(image_name)

X_test = np.array(X_test)
y_test = np.array(y_test)

# Normaliser les données test avec le même scaler
X_test_scaled = scaler.transform(X_test)

# --- PRÉDICTIONS ---
y_pred = rf.predict(X_test_scaled)

# --- CALCUL DE L'ERREUR ---
mae_test = mean_absolute_error(y_test, y_pred)

# --- AFFICHER LES RÉSULTATS ---
print(f"Erreur Absolue Moyenne (MAE) sur l'ensemble test : {mae_test:.2f}")

# Afficher quelques résultats détaillés
for img, actual, pred in zip(image_names[:10], y_test[:10], y_pred[:10]):
    print(f"Image: {img} | Vérité Terrain: {actual} | Prédiction: {round(pred)}")


Erreur Absolue Moyenne (MAE) sur l'ensemble test : 5.17
Image: img6.jpg | Vérité Terrain: 11 | Prédiction: 9
Image: t3i21.jpeg | Vérité Terrain: 14 | Prédiction: 12
Image: Groupe1_Image4.jpg | Vérité Terrain: 12 | Prédiction: 11
Image: Groupe5_image11.jpeg | Vérité Terrain: 13 | Prédiction: 12
Image: Groupe5_image17.jpeg | Vérité Terrain: 13 | Prédiction: 14
Image: Groupe1_Image1.jpg | Vérité Terrain: 11 | Prédiction: 9
Image: img4.jpg | Vérité Terrain: 37 | Prédiction: 13
Image: t3i20.png | Vérité Terrain: 2 | Prédiction: 5
Image: Groupe5_image13.jpeg | Vérité Terrain: 3 | Prédiction: 14
Image: t3i14.jpeg | Vérité Terrain: 4 | Prédiction: 11


In [6]:
import os
import json
import cv2
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import LeaveOneOut
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error

# --- CHARGER LES DONNÉES JSON ---
json_path = "gt.json"
with open(json_path, "r") as f:
    data = json.load(f)

# Créer un dictionnaire {image_name: actual_count}
ground_truth = {item["image"]: item["actual_count"] for item in data["data"]}

# --- NOUVELLE EXTRACTION DES CARACTÉRISTIQUES ---
def extract_histogram_features(image_path, num_bins=256, image_height=1024):
    img = cv2.imread(image_path)
    if img is None:
        return None  # Gestion des erreurs de chargement

    # Redimensionner l’image
    img_resized = cv2.resize(img, (1024, image_height))
    gray = cv2.cvtColor(img_resized, cv2.COLOR_BGR2GRAY)

    # Détection des contours
    edges = cv2.Canny(gray, 50, 150, apertureSize=3)

    # Détection des lignes
    lines = cv2.HoughLinesP(edges, 1, np.pi/180, 100, minLineLength=100, maxLineGap=10)

    # Initialiser l'histogramme
    histogram = np.zeros(num_bins)

    if lines is not None:
        for line in lines:
            for x1, y1, x2, y2 in line:
                center_y = (y1 + y2) // 2
                bin_index = center_y // 4  # Réduction de la résolution
                histogram[bin_index] += 1

    return histogram

# --- EXTRAIRE LES DONNÉES POUR L'ENTRAÎNEMENT ---
train_folder = "data/train"

X, y = [], []
for image_name in os.listdir(train_folder):
    if image_name in ground_truth:
        image_path = os.path.join(train_folder, image_name)
        features = extract_histogram_features(image_path)
        if features is not None:
            X.append(features)
            y.append(ground_truth[image_name])

X = np.array(X)
y = np.array(y)

# --- NORMALISATION ---
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# --- ENTRAÎNEMENT DU MODÈLE ---
loo = LeaveOneOut()
errors_rf = []

rf = RandomForestRegressor(n_estimators=1000, random_state=42, warm_start=True)

for train_idx, test_idx in loo.split(X_scaled):
    X_train, X_test = X_scaled[train_idx], X_scaled[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]

    if len(X_train) > 1:
        rf.fit(X_train, y_train)

    y_pred = rf.predict(X_test)
    errors_rf.append(abs(y_test[0] - y_pred[0]))

# --- CALCUL DE LA MAE ---
mean_error_rf = np.mean(errors_rf)
print(f"Erreur Absolue Moyenne (MAE) sur train : {mean_error_rf:.2f}")


/usr/local/lib/python3.11/dist-packages/sklearn/ensemble/_forest.py:466: UserWarning: Warm-start fitting without increasing n_estimators does not fit new trees.
  warn(
/usr/local/lib/python3.11/dist-packages/sklearn/ensemble/_forest.py:466: UserWarning: Warm-start fitting without increasing n_estimators does not fit new trees.
  warn(
/usr/local/lib/python3.11/dist-packages/sklearn/ensemble/_forest.py:466: UserWarning: Warm-start fitting without increasing n_estimators does not fit new trees.
  warn(
/usr/local/lib/python3.11/dist-packages/sklearn/ensemble/_forest.py:466: UserWarning: Warm-start fitting without increasing n_estimators does not fit new trees.
  warn(
/usr/local/lib/python3.11/dist-packages/sklearn/ensemble/_forest.py:466: UserWarning: Warm-start fitting without increasing n_estimators does not fit new trees.
  warn(
/usr/local/lib/python3.11/dist-packages/sklearn/ensemble/_forest.py:466: UserWarning: Warm-start fitting without increasing n_estimators does not fit new t

Erreur Absolue Moyenne (MAE) sur train : 1.76


In [7]:
# --- TEST DU MODÈLE ---
test_folder = "data/test"

X_test, y_test, image_names = [], [], []
for image_name in os.listdir(test_folder):
    if image_name in ground_truth:
        image_path = os.path.join(test_folder, image_name)
        features = extract_histogram_features(image_path)
        if features is not None:
            X_test.append(features)
            y_test.append(ground_truth[image_name])
            image_names.append(image_name)

X_test = np.array(X_test)
y_test = np.array(y_test)

# Normaliser les données test
X_test_scaled = scaler.transform(X_test)

# --- PRÉDICTIONS ---
y_pred = rf.predict(X_test_scaled)

# --- CALCUL DE L'ERREUR ---
mae_test = mean_absolute_error(y_test, y_pred)

# --- AFFICHER LES RÉSULTATS ---
print(f"Erreur Absolue Moyenne (MAE) sur test : {mae_test:.2f}")

# Afficher quelques résultats détaillés
for img, actual, pred in zip(image_names[:10], y_test[:10], y_pred[:10]):
    print(f"Image: {img} | Vérité Terrain: {actual} | Prédiction: {round(pred)}")


Erreur Absolue Moyenne (MAE) sur test : 3.89
Image: img6.jpg | Vérité Terrain: 11 | Prédiction: 11
Image: t3i21.jpeg | Vérité Terrain: 14 | Prédiction: 13
Image: Groupe1_Image4.jpg | Vérité Terrain: 12 | Prédiction: 10
Image: Groupe5_image11.jpeg | Vérité Terrain: 13 | Prédiction: 11
Image: Groupe5_image17.jpeg | Vérité Terrain: 13 | Prédiction: 14
Image: Groupe1_Image1.jpg | Vérité Terrain: 11 | Prédiction: 9
Image: img4.jpg | Vérité Terrain: 37 | Prédiction: 16
Image: t3i20.png | Vérité Terrain: 2 | Prédiction: 5
Image: Groupe5_image13.jpeg | Vérité Terrain: 3 | Prédiction: 11
Image: t3i14.jpeg | Vérité Terrain: 4 | Prédiction: 9
